# Model creation

In [ ]:
import os

ROOT_DIR = os.getcwd()
PROJECT_NAME = 'isnet'

%cd {ROOT_DIR}

In [ ]:
TARBALL_NAME = 'isnet-general-use.tar.gz'
if os.path.exists(TARBALL_NAME):
    !rm {TARBALL_NAME}
!rm -rf my_model/code/.ipynb_checkpoints
!rm -rf my_model/code/__pycache__
!tar -czvf {TARBALL_NAME} my_model

## Move the tarball to S3

In [ ]:
import sagemaker

KEY_PREFIX = 'isnet'
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path=TARBALL_NAME, key_prefix=KEY_PREFIX)

# Endpoint configuration creation

In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

# Create an empty ServerlessInferenceConfig object to use default values
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=1024,
    max_concurrency=20
)

# Endpoint creation and invocation

In [ ]:
import boto3
from sagemaker.pytorch.model import PyTorchModel
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

%cd {ROOT_DIR}/my_model/code
model = PyTorchModel(
    model_data='s3://' + sagemaker_session.default_bucket() + f'{KEY_PREFIX}/{TARBALL_NAME}',
    role=role,
    framework_version='1.10.0',
    py_version='py3',
    source_dir='.',
    entry_point='inference.py'
)

In [ ]:
serverless_predictor = model.deploy(serverless_inference_config=serverless_config)

In [ ]:
serverless_predictor.endpoint_name

# API Gateway connection